In [32]:
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from dotenv import load_dotenv
import os
import seaborn as sns

tqdm.pandas()
load_dotenv()

CR_ID = "e066e8fe16ed43b7b59c1c46986bd206"
INSTRUMENT_ID = "spx"
INTERVAL = "1min" # OneMinute,FiveMinutes
start_date="2020-01-06"
end_date="2023-08-10"

# API_URL=f"https://candle.etoro.com/candles/asc.json/{INTERVAL}/20000/{INSTRUMENT_ID}?client_request_id={CR_ID}"
API_URL=f"https://api.twelvedata.com/time_series?start_date={start_date}&end_date={end_date}&symbol={INSTRUMENT_ID}&interval={INTERVAL}&apikey={CR_ID}"

In [33]:
import requests
import pandas as pd
import json

# Make the API request
response = requests.get(API_URL)

# Check if the request was successful
if response.status_code == 200:
    api_data = response.json()

    if api_data["status"] == 'ok':
        # Extract necessary data from the JSON
        symbol = api_data['meta']['symbol']
        interval = api_data['meta']['interval']
        values = api_data['values']
        
        # Create a list of dictionaries to convert to a dataframe
        data_list = []
        for value in values:
            data_list.append({
                'datetime': value['datetime'],
                'open': float(value['open']),
                'high': float(value['high']),
                'low': float(value['low']),
                'close': float(value['close']),
                'volume': int(value['volume'])
            })
        
        # Create a dataframe from the list of dictionaries
        df = pd.DataFrame(data_list)
        
        # Display the dataframe
        print(df)

        # cache it
        df.to_json(f"./ts/{start_date}-{end_date}-{INSTRUMENT_ID}-{INTERVAL}.json")
    else:
        print("API request failed with:", api_data["message"])
else:
    print("API request failed with status code:", response.status_code)


                 datetime        open        high         low       close  \
0     2023-08-09 15:59:00  4467.72021  4470.29980  4466.16992  4467.91016   
1     2023-08-09 15:58:00  4468.91016  4469.62012  4467.68018  4467.68018   
2     2023-08-09 15:57:00  4469.22021  4470.47021  4468.79980  4468.95020   
3     2023-08-09 15:56:00  4466.85986  4470.00977  4465.52002  4469.16992   
4     2023-08-09 15:55:00  4471.14990  4471.14990  4466.37988  4466.45020   
...                   ...         ...         ...         ...         ...   
4995  2023-07-24 10:43:00  4552.24023  4553.52002  4551.95020  4553.52002   
4996  2023-07-24 10:42:00  4553.02979  4554.08984  4551.56982  4552.14014   
4997  2023-07-24 10:41:00  4553.14014  4553.35010  4552.91992  4553.00000   
4998  2023-07-24 10:40:00  4551.85986  4553.29980  4551.85986  4553.14990   
4999  2023-07-24 10:39:00  4552.27002  4552.45020  4551.20996  4551.52002   

        volume  
0     25993583  
1     21912423  
2     19452902  
3     2